# Allow for image level parameter input: added to *digipath_toolkit.py*

## *get_patch_location_array_for_image_level(run_parameters)*

## *get_patch_locations_preview_imagefor_image_level(run_parameters)*

In [ ]:
import time
nb_start_time = time.time()

import os
import sys

from collections import OrderedDict
import argparse

import numpy as np
import pandas as pd
import yaml

from skimage.filters import threshold_otsu
from skimage.color import rgb2lab

from PIL import ImageDraw
from PIL import TiffImagePlugin as tip

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

DEFAULT_THUMBNAIL_DIVISOR = 20

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

In [ ]:
#                         test data file for threshold_otsu
data_dir = '../../DigiPath_MLTK_data/Aperio'
# image_file_name = 'CMU-2.svs'
# image_file_name = 'JP2K-33003-2.svs'
image_file_name = 'CMU-1-Small-Region.svs'

run_parameters = dict()
run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = DEFAULT_THUMBNAIL_DIVISOR
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224
run_parameters['threshold'] = 0
run_parameters['border_color'] = 'blue'

lineprint_level_sizes_dict(run_parameters['wsi_filename'])

In [ ]:
level_sizes_dict = get_level_sizes_dict(run_parameters['wsi_filename'])

n_levels = level_sizes_dict['level_count']
for im_lvl in range(0, n_levels):
    run_parameters['image_level'] = im_lvl
    run_parameters['thumbnail_divisor'] = DEFAULT_THUMBNAIL_DIVISOR
    
    t1 = time.time()

    patch_location_array = get_patch_location_array_for_image_level(run_parameters)
    print('\nimage_level = %i,\nnumber of patches = %i'%(im_lvl, len(patch_location_array)))

    run_parameters['thumbnail_divisor'] = run_parameters['thumbnail_divisor'] // (2*run_parameters['image_level'] + 1)
    
    mask_im, prev_im, patch_array = get_patch_locations_preview_imagefor_image_level(run_parameters)

    print('image_level = %i,\nnumber of patches = %i'%(im_lvl, len(patch_array)))
    print('cell run time: %0.3f'%(time.time() - t1))
    print('thumb image size:', prev_im.size, 'thumbnail_divisor', run_parameters['thumbnail_divisor'])
    
    display(prev_im)